In [ ]:
#unfiltered dataset
import os
import random
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# Step 1: Paths
DATA_DIR = "/content/drive/MyDrive/data"
MODEL_PATH = "writer_id_model.pth"

# Step 2: Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("🖥️ Using device:", device)

# Step 3: Model with pretrained weights
def build_model(num_classes):
    model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)  # For grayscale
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model.to(device)

# Step 4: Dataset Preparation — per-writer split
def split_dataset_per_writer(root_dir, transform, split_ratio=0.8):
    class_names = sorted(os.listdir(root_dir))
    class_to_idx = {cls_name: i for i, cls_name in enumerate(class_names)}
    train_samples, test_samples = [], []

    for writer in class_names:
        writer_dir = os.path.join(root_dir, writer)
        images = [os.path.join(writer_dir, f) for f in os.listdir(writer_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]
        random.shuffle(images)
        split = int(len(images) * split_ratio)
        train_samples += [(img, class_to_idx[writer]) for img in images[:split]]
        test_samples  += [(img, class_to_idx[writer]) for img in images[split:]]

    return CustomImageDataset(train_samples, transform), CustomImageDataset(test_samples, transform), class_names, test_samples

# Step 5: Custom Dataset
class CustomImageDataset(torch.utils.data.Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert("L")
        if self.transform:
            image = self.transform(image)
        return image, label

# Step 6: Train
def train_model(train_loader, num_classes, epochs=50, lr=1e-4):
    model = build_model(num_classes)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"📉 Epoch {epoch+1} Loss: {total_loss:.4f}")

    torch.save(model.state_dict(), MODEL_PATH)
    print("✅ Model saved:", MODEL_PATH)
    return model

# Step 7: Evaluate
def evaluate_model(model, test_loader):
    model.eval()
    correct = total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            preds = model(images).argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    print(f"✅ Test Accuracy: {correct / total * 100:.2f}%")

# Step 8: Predict
def predict_random_test_image(model, test_samples, class_names, transform):
    path, true_label = random.choice(test_samples)
    img = Image.open(path).convert("L")
    img_tensor = transform(img).unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        output = model(img_tensor)
        pred_label = torch.argmax(output, 1).item()
    print(f"\n🖼️ Predicted Writer: {class_names[pred_label]}")
    print(f"✅ Actual Writer:    {class_names[true_label]}")
    print(f"📂 File: {os.path.basename(path)}")

# Step 9: Run
def main():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    train_ds, test_ds, class_names, test_samples = split_dataset_per_writer(DATA_DIR, transform)
    train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)   # Increased batch size
    test_loader  = DataLoader(test_ds, batch_size=16, shuffle=False)

    model = train_model(train_loader, num_classes=len(class_names), epochs=50, lr=1e-4)
    evaluate_model(model, test_loader)
    predict_random_test_image(model, test_samples, class_names, transform)

main()


🖥️ Using device: cuda


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 177MB/s]
Epoch 1/50: 100%|██████████| 53/53 [01:10<00:00,  1.33s/it]


📉 Epoch 1 Loss: 330.9981


Epoch 2/50: 100%|██████████| 53/53 [01:07<00:00,  1.27s/it]


📉 Epoch 2 Loss: 278.0735


...

📉 Epoch 48 Loss: 60.5348


Epoch 49/50: 100%|██████████| 53/53 [01:07<00:00,  1.27s/it]


📉 Epoch 49 Loss: 58.8951


Epoch 50/50: 100%|██████████| 53/53 [01:07<00:00,  1.27s/it]


📉 Epoch 50 Loss: 60.4404
✅ Model saved: writer_id_model.pth
✅ Test Accuracy: 31.73%

🖼️ Predicted Writer: 155
✅ Actual Writer:    155
📂 File: c03-081f.png


In [ ]:
#filtered dataset
import os
import random
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, random_split
from torchvision import transforms, models
from tqdm import tqdm

class FilteredWriterDataset(Dataset):
    def __init__(self, root_dir, transform=None, min_images=10):
        self.transform = transform
        self.class_names = [d for d in os.listdir(root_dir) if len(os.listdir(os.path.join(root_dir, d))) >= min_images]
        self.class_names.sort()
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.class_names)}

        self.samples = []
        for cls in self.class_names:
            folder = os.path.join(root_dir, cls)
            images = [f for f in os.listdir(folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
            self.samples.extend([(os.path.join(folder, img), self.class_to_idx[cls]) for img in images])

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        path, label = self.samples[idx]
        image = Image.open(path).convert("L")
        if self.transform:
            image = self.transform(image)
        return image, label

class CosineClassifier(nn.Module):
    def __init__(self, in_features, num_classes):
        super().__init__()
        self.weight = nn.Parameter(torch.Tensor(num_classes, in_features))
        nn.init.xavier_uniform_(self.weight)

    def forward(self, x):
        x = F.normalize(x)
        w = F.normalize(self.weight)
        return torch.matmul(x, w.t()) * 10


class WriterClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.backbone = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.backbone.fc = nn.Identity()
        self.classifier = CosineClassifier(in_features=512, num_classes=num_classes)

    def forward(self, x):
        features = self.backbone(x)
        return self.classifier(features)

def train_model(model, loader, criterion, optimizer, device, epochs=70):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in tqdm(loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"📉 Epoch {epoch+1} Loss: {total_loss:.4f}")


def evaluate(model, loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    acc = correct / total * 100
    print(f"🎯 Total Accuracy: {acc:.2f}%")

def run_pipeline(data_dir):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    dataset = FilteredWriterDataset(data_dir, transform, min_images=5)
    print(f"🧾 Writers after filtering: {len(dataset.class_names)}")

    # Balanced sampler
    labels = [label for _, label in dataset.samples]
    class_count = torch.bincount(torch.tensor(labels))
    weights = 1.0 / class_count[labels]
    #sampler = WeightedRandomSampler(weights, len(weights))

    # Train/test split
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_ds, test_ds = random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_ds, batch_size=16, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=16, shuffle=False)


    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = WriterClassifier(num_classes=len(dataset.class_names)).to(device)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    train_model(model, train_loader, criterion, optimizer, device, epochs=50)
    evaluate(model, test_loader, device)

    torch.save(model.state_dict(), "writer_classifier_cosine.pth")
    print("✅ Model saved.")

run_pipeline("/content/drive/MyDrive/data")  # or your actual dataset path


🧾 Writers after filtering: 94


Epoch 1/50: 100%|██████████| 34/34 [00:42<00:00,  1.25s/it]


📉 Epoch 1 Loss: 152.2293


Epoch 2/50: 100%|██████████| 34/34 [00:42<00:00,  1.26s/it]


📉 Epoch 2 Loss: 133.7400


...

📉 Epoch 48 Loss: 27.5545


Epoch 49/50: 100%|██████████| 34/34 [00:42<00:00,  1.25s/it]


📉 Epoch 49 Loss: 27.3119


Epoch 50/50: 100%|██████████| 34/34 [00:42<00:00,  1.24s/it]


📉 Epoch 50 Loss: 27.4303
🎯 Total Accuracy: 94.07%
✅ Model saved.
